In [2]:
_EPSILON = 1e-08

import numpy as np
import pandas as pd
import random
import os

In [51]:
df_s  = pd.read_csv('./data/processed/after imputation/baseline.csv')
df_t  = pd.read_csv('./data/processed/after imputation/temporal.csv')
df_t  = df_t.drop_duplicates(subset=['New ID', 'Days Since Diagnosis'], keep='last').reset_index(drop=True)

df_t1  = pd.read_csv('./data/processed/after imputation/repeat_psa.csv')
df_t2  = pd.read_csv('./data/processed/after imputation/repeat_mri.csv')
df_t3  = pd.read_csv('./data/processed/after imputation/repeat_biopsy.csv')

df_t1  = df_t1.drop_duplicates(subset=['New ID', 'Days Since Diagnosis'], keep='last').reset_index(drop=True)
df_t2  = df_t2.drop_duplicates(subset=['New ID', 'Days Since Diagnosis'], keep='last').reset_index(drop=True)
df_t3  = df_t3.drop_duplicates(subset=['New ID', 'Days Since Diagnosis'], keep='last').reset_index(drop=True)

### Compute Delta

In [52]:
first = df_t.drop_duplicates(subset=['New ID',], keep='first')[['Days Since Diagnosis']]
diff  = df_t['Days Since Diagnosis'].diff()
diff[first.index] = first['Days Since Diagnosis']

time  = df_t[['New ID', 'Days Since Diagnosis']]

df_t['Days Since Diagnosis'] = diff
df_t = df_t.rename(columns={'Days Since Diagnosis': 'Delta'})

In [53]:
first = df_t1.drop_duplicates(subset=['New ID',], keep='first')[['Days Since Diagnosis']]
diff  = df_t1['Days Since Diagnosis'].diff()
diff[first.index] = first['Days Since Diagnosis']

time1 = df_t1[['New ID', 'Days Since Diagnosis']]

df_t1['Days Since Diagnosis'] = diff
df_t1 = df_t1.rename(columns={'Days Since Diagnosis': 'Delta'})

In [54]:
first = df_t2.drop_duplicates(subset=['New ID',], keep='first')[['Days Since Diagnosis']]
diff  = df_t2['Days Since Diagnosis'].diff()
diff[first.index] = first['Days Since Diagnosis']

time2 = df_t2[['New ID', 'Days Since Diagnosis']]

df_t2['Days Since Diagnosis'] = diff
df_t2 = df_t2.rename(columns={'Days Since Diagnosis': 'Delta'})

In [55]:
first = df_t3.drop_duplicates(subset=['New ID',], keep='first')[['Days Since Diagnosis']]
diff  = df_t3['Days Since Diagnosis'].diff()
diff[first.index] = first['Days Since Diagnosis']

time3 = df_t3[['New ID', 'Days Since Diagnosis']]

df_t3['Days Since Diagnosis'] = diff
df_t3 = df_t3.rename(columns={'Days Since Diagnosis': 'Delta'})

In [56]:
feat_static = [
    'Exact age at diagnosis', 'Number of negative biopsies before diagnosis',
    'Number of MRI-visible lesions', 'Ethnicity', 'Family History of Prostate Cancer',
    'CPG', 'PI-RADS score', 'STRATCANS (simplified)',
]

feat_timevarying1 = [
     'Delta', 'Repeat PSA'
]

feat_timevarying2 = [
     'Delta', 'Repeat MRI PRECISE Scoring', 'Repeat MRI Stage','Repeat MRI Volume', 'Repeat MRI PSAd'
]

feat_timevarying3 = [
     'Delta', 'Repeat Biopsy Core Total', 'Repeat Biopsy Core Positive', 
#      'Repeat Biopsy Primary Gleason', 'Repeat Biopsy Secondary Gleason', 
     'Repeat Biopsy Grade Group'
]

feat_label = [
    'Coding.3', 'Days since diagnosis.3'
]


# xt_bin_list = []
# xt_con_list = [0,1,2,3,4,5,6,7,8,9]


data_s = df_s[['New ID'] + feat_static]

data_t1 = df_t1[['New ID'] + feat_timevarying1]
data_t2 = df_t2[['New ID'] + feat_timevarying2]
data_t3 = df_t3[['New ID'] + feat_timevarying3]

label   = df_s[['New ID'] + feat_label]

In [57]:
id_list = np.unique(data_t1['New ID'].unique().tolist() + data_t2['New ID'].unique().tolist() + data_t3['New ID'].unique().tolist())

data_s  = data_s[data_s['New ID'].isin(id_list)].reset_index(drop=True)
label   = label[label['New ID'].isin(id_list)].reset_index(drop=True)

In [63]:
from sklearn.preprocessing import MinMaxScaler

scaler_s                 = MinMaxScaler()
scaler_t1                 = MinMaxScaler()
scaler_t2                 = MinMaxScaler()
scaler_t3                 = MinMaxScaler()

data_t1.loc[data_t1['Delta'] == 0, 'Delta'] = 1. #0 makes -inf!
data_t1['Delta']          = np.log(data_t1['Delta'])  ### this is to deal with very skewed deltas...

data_t2.loc[data_t2['Delta'] == 0, 'Delta'] = 1. #0 makes -inf!
data_t2['Delta']          = np.log(data_t2['Delta'])  ### this is to deal with very skewed deltas...

data_t3.loc[data_t3['Delta'] == 0, 'Delta'] = 1. #0 makes -inf!
data_t3['Delta']          = np.log(data_t3['Delta'])  ### this is to deal with very skewed deltas...


data_s[feat_static]      = scaler_s.fit_transform(data_s[feat_static])
data_t1[feat_timevarying1] = scaler_t1.fit_transform(data_t1[feat_timevarying1])
data_t2[feat_timevarying2] = scaler_t2.fit_transform(data_t2[feat_timevarying2])
data_t3[feat_timevarying3] = scaler_t3.fit_transform(data_t3[feat_timevarying3])

/home/vdslab/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/vdslab/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/vdslab/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [72]:
xs     = np.asarray(data_s)[:, 1:]

In [92]:
grouped     = data_t1.groupby(by=['New ID'])
max_length  = grouped.count()['Delta'].max() #max_length = 48

xt1         = np.zeros([len(id_list), max_length, len(feat_timevarying1)])
for i, pid in enumerate(id_list):
    tmp    = np.asarray(grouped.get_group(pid))[:, 1:]
    xt1[i, :np.shape(tmp)[0], :] = tmp
    
    
grouped    = time1.groupby(by=['New ID'])

t1          = np.zeros([len(id_list), max_length])
for i, pid in enumerate(id_list):
    tmp    = np.asarray(grouped.get_group(pid))[:, 1:]
    t1[i, :np.shape(tmp)[0]] = tmp[:,0]

In [94]:
grouped     = data_t2.groupby(by=['New ID'])
max_length  = grouped.count()['Delta'].max() #max_length = 48

xt2         = np.zeros([len(id_list), max_length, len(feat_timevarying2)])
for i, pid in enumerate(id_list):
    tmp    = np.asarray(grouped.get_group(pid))[:, 1:]
    xt2[i, :np.shape(tmp)[0], :] = tmp
    
    
grouped    = time2.groupby(by=['New ID'])

t2          = np.zeros([len(id_list), max_length])
for i, pid in enumerate(id_list):
    tmp    = np.asarray(grouped.get_group(pid))[:, 1:]
    t2[i, :np.shape(tmp)[0]] = tmp[:,0]

In [96]:
grouped     = data_t3.groupby(by=['New ID'])
max_length  = grouped.count()['Delta'].max() 

xt3         = np.zeros([len(id_list), max_length, len(feat_timevarying3)])
for i, pid in enumerate(id_list):
    tmp    = np.asarray(grouped.get_group(pid))[:, 1:]
    xt3[i, :np.shape(tmp)[0], :] = tmp
    
    
grouped    = time3.groupby(by=['New ID'])

t3          = np.zeros([len(id_list), max_length])
for i, pid in enumerate(id_list):
    tmp    = np.asarray(grouped.get_group(pid))[:, 1:]
    t3[i, :np.shape(tmp)[0]] = tmp[:, 0]

In [85]:
y   = np.asarray(label[['Coding.3']])
tte = np.asarray(label[['Days since diagnosis.3']])

In [105]:
import pickle

np.savez(
    './data/processed/final/version2 (separate)/dataset.npz',
    data_xs   = xs,
    
    data_xt1   = xt1,
    data_xt2   = xt2,
    data_xt3   = xt3,
    
    data_time1 = t1,
    data_time2 = t2,
    data_time3 = t3,
    
    data_y    = y,
    data_tte  = tte,
    feat_static = feat_static,
    
    feat_timevarying1 = feat_timevarying1,
    feat_timevarying2 = feat_timevarying2,
    feat_timevarying3 = feat_timevarying3,
)

pickle.dump(scaler_s, open('./data/processed/final/version2 (separate)/scaler_s.pkl', 'wb'))
pickle.dump(scaler_t1, open('./data/processed/final/version2 (separate)/scaler_t1.pkl', 'wb'))
pickle.dump(scaler_t2, open('./data/processed/final/version2 (separate)/scaler_t2.pkl', 'wb'))
pickle.dump(scaler_t3, open('./data/processed/final/version2 (separate)/scaler_t3.pkl', 'wb'))
